# tkinter, Canvas, matplotlib y la libreria psutil
La libreria `psutil` permite obtener información del CPU y de todos los perifericos en tiempos real. Vamos a utilizar esta libreria para obtener información real sobre un sistema para poder mostrarla en un ambiente gráfico y posteriormente combinarla con un grafico de maptlotlib insertado dentro de un Canvas especial llamado FigureCanvasTkAgg

In [3]:
# pip install psutil
import psutil

print("Uso del CPU(%):", psutil.cpu_percent())
print("Velocidad del CPU(Ghz)", psutil.cpu_freq().current)
print("Total de memoria(GB):", psutil.virtual_memory().total/1e9)
print("Memoria en uso(%):", psutil.virtual_memory().percent)
print("Interface Ethernet IP:", psutil.net_if_addrs()['Ethernet'][1].address)

Uso del CPU(%): 19.2
Velocidad del CPU(Ghz) 2500.0
Total de memoria(GB): 12.307968
Memoria en uso(%): 41.5
Interface Ethernet IP: 169.254.180.136


Vamos a combinar la informacion que nos da la librería `psutil` con algun indicador gráfico de tkinter que permita visualizar el uso de la CPU (`ttk.Progressbar`).

Adicionalmente, utilizaremos el método `after` de una ventana (`root.after(ms, func)`) para invocar a una función luego de un tiempo y utilizando un encadenamiento de funciones lograremos actualizar la información de forma continua.

In [2]:
# MONITOR DE USO DE CPU Y MEMORIA (PROGRESSBAR)
import tkinter as tk
from tkinter import ttk
import psutil

root = tk.Tk()
root.title("CPU Monitor")
root.geometry("250x150")
root.config(bg='black')

valCPU = tk.DoubleVar(value=0.0)
valMEM = tk.DoubleVar(value=0.0)

frm = tk.Frame(root, bg='black')
frm.pack()

lblCPU_Usage = tk.Label(frm, text="   CPU Usage: ", bg='black', fg='white')     
lblCPU_Usage_percent = tk.Label(frm, text="{:.1f}%".format(valCPU.get()), bg='black', fg='white')
barCPU_Usage = ttk.Progressbar(frm, length=200, variable=valCPU)
lblMemory_Usage = tk.Label(frm, text="   Memory Usage: ", bg='black', fg='white')
lblMemory_Usage_percent = tk.Label(frm, text="{:.1f}%".format(valMEM.get()), bg='black', fg='white')
barMemory_Usage = ttk.Progressbar(frm, length=200, variable=valMEM)

lblCPU_Usage.grid(row=0, column=0, pady=10, sticky='w')
lblCPU_Usage_percent.grid(row=0, column=1) 
barCPU_Usage.grid(row=1, column=0, columnspan=2) 
lblMemory_Usage.grid(row=2, column=0, pady=10, sticky='w')
lblMemory_Usage_percent.grid(row=2, column=1)
barMemory_Usage.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

def read_statistics():
    read_cpu_data()
    
def read_cpu_data():
    valCPU.set(psutil.cpu_percent())
    lblCPU_Usage_percent.config(text="{:.1f}%".format(valCPU.get()))
    
    memory_data = psutil.virtual_memory()
    valMEM.set(memory_data.percent)
    lblMemory_Usage_percent.config(text="{:.1f}%".format(valMEM.get()))
    
    root.after(1000, read_statistics)
    
root.after(100, read_statistics)
root.mainloop()

Podemos incluir un gráfico de matplotlib en un GUI hecho en tkinter utiliznado un `Canvas` especial llamado `FigureCanvasTkAgg`

In [1]:
# MONITOR GRAFICO DE USO DE CPU (PLOT)
from tkinter import *
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import psutil

global data_cpu
data_cpu = [0] * 100

root = Tk()
root.title("CPU Monitor")
#root.config(bg='black')
root.geometry('500x400')

lab = Label(root, text='CPU Monitor Ploting', bg='white')   #, bg='black', fg='white'
lab.pack(fill=X)

fig = Figure()      #facecolor='black'

ax = fig.add_subplot(111)    # facecolor='black'
ax.set_xlabel('Time')
ax.set_ylabel('% Usage')

graph = FigureCanvasTkAgg(fig, master=root)
graph.get_tk_widget().pack(side='top', fill='both', expand=True)

def update_graph():
    plotter()

def plotter():
    global data_cpu
    
    data_cpu.pop(0)
    data_cpu.append(psutil.cpu_percent())
    
    ax.cla()
    ax.grid()
    ax.set_ylim(0, 100)
    #ax.set_yticks([300])
    ax.set_yticklabels([0, 20, 40, 60, 80, 100])   #fontsize=6
    #ax.set_xticks([300])
    ax.plot(data_cpu, color='blue')
    graph.draw()
    root.after(100, update_graph)
        
root.after(100, update_graph)
root.mainloop()